In [1]:
import numpy as np

from graph_emb.classify import read_node_label, Classifier
from graph_emb import DeepWalk
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import networkx as nx
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN

In [2]:
def evaluate_embeddings(embeddings):
    X, Y = read_node_label('../data/wiki/wiki_labels.txt')
    tr_frac = 0.8
    print("Training classifier using {:.2f}% nodes...".format(
        tr_frac * 100))
    clf = Classifier(embeddings=embeddings, clf=LogisticRegression())
    clf.split_train_evaluate(X, Y, tr_frac)


def plot_embeddings(embeddings,):
    X, Y = read_node_label('../data/wiki/wiki_labels.txt')

    emb_list = []
    for k in X:
        emb_list.append(embeddings[k])
    emb_list = np.array(emb_list)

    model = TSNE(n_components=2)
    node_pos = model.fit_transform(emb_list)

    color_idx = {}
    for i in range(len(X)):
        color_idx.setdefault(Y[i][0], [])
        color_idx[Y[i][0]].append(i)

    for c, idx in color_idx.items():
        plt.scatter(node_pos[idx, 0], node_pos[idx, 1], label=c)
    plt.legend()
    plt.show()

In [3]:
# read contrusted graph
G = nx.read_gpickle("./our_data/graph.gpickle")

In [4]:
# 序列长度，xxx，并行worker数量
model = DeepWalk(G, walk_length=10, num_walks=80, workers=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.6s finished


In [5]:
model.train(window_size=5, iter=3) 
embeddings = model.get_embeddings()

Learning embedding vectors...
Learning embedding vectors done!


In [6]:
train_X = []
train_X_id = []

for k, v in embeddings.items():
    train_X.append(v)
    train_X_id.append(v)

train_X = np.array(train_X)
clustering = DBSCAN().fit(train_X)
# evaluate_embeddings(embeddings)
# plot_embeddings(embeddings)